# Phase 1: Extract

In [ ]:
import pandas as pd
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='your password',
    database='ecommerce_db'
)
cursor = conn.cursor()

# Extract data from CSV files
products_df = pd.read_csv('~/Documents/ETL_Project/products_data.csv')
orders_df = pd.read_csv('~/Documents/ETL_Project/orders_data.csv')
users_df = pd.read_csv('~/Documents/ETL_Project/users_data.csv')

# Extract data from Excel files
payments_df = pd.read_excel('~/Documents/ETL_Project/payments_data.xlsx', engine='openpyxl')
reviews_df = pd.read_excel('~/Documents/ETL_Project/reviews_data.xlsx', engine='openpyxl')


## What’s Happening Here?
* We start by importing the necessary libraries: pandas for data manipulation and mysql.connector to connect to the MySQL database.
* We then establish a connection to the MySQL database using the mysql.connector.connect method.
* The pd.read_csv and pd.read_excel functions are used to extract data from CSV and Excel files, respectively. This data is loaded into DataFrames, which are versatile and powerful data structures in Python that allow us to easily manipulate and analyze the data.

# Phase 2: Transform

In [3]:
# Fill missing values in each dataframe as needed
products_df = products_df.fillna({
    'Description': 'No description available',
    'Category': 'Miscellaneous'
})

orders_df = orders_df.fillna({
    'TotalAmount': 0.0
})

payments_df = payments_df.fillna({
    'PaymentMethod': 'Unknown',
    'Amount': 0.0,
    'PaymentDate': pd.Timestamp('1970-01-01')
})

reviews_df = reviews_df.fillna({
    'ReviewText': 'no review'
})

# Handling invalid emails in the Users DataFrame
def generate_unique_email(index):
    return f'unknown_{index}@example.com'

users_df['Email'] = users_df.apply(lambda row: row['Email'] if '@' in row['Email'] and '.' in row['Email'] else generate_unique_email(row.name), axis=1)

# Ensure all prices are numeric
products_df['Price'] = pd.to_numeric(products_df['Price'], errors='coerce').fillna(0.0).round(2)

# Ensure all stock quantities are numeric
products_df['StockQuantity'] = pd.to_numeric(products_df['StockQuantity'], errors='coerce').fillna(0).astype(int)

# Standardize and convert date formats to string
orders_df['OrderDate'] = pd.to_datetime(orders_df['OrderDate'], errors='coerce', format='%Y-%m-%d').fillna(pd.Timestamp('1970-01-01'))
orders_df['OrderDate'] = orders_df['OrderDate'].dt.strftime('%Y-%m-%d')

payments_df['PaymentDate'] = pd.to_datetime(payments_df['PaymentDate'], errors='coerce', format='%Y-%m-%d').fillna(pd.Timestamp('1970-01-01'))
payments_df['PaymentDate'] = payments_df['PaymentDate'].dt.strftime('%Y-%m-%d')


## What’s Happening Here?
* Filling Missing Values: We use the fillna method to replace missing values with default values. For example, if the Description or Category fields are missing in the products_df DataFrame, we fill them with 'No description available' and 'Miscellaneous', respectively.
* Handling Invalid Emails: We define a function generate_unique_email to create a placeholder email for records with invalid or missing email addresses.
* Ensuring Numeric Data: We convert the Price and StockQuantity fields in products_df to numeric values. Any non-numeric data is replaced with a default value (0.0 for prices, 0 for stock quantities).
* Standardizing Dates: Dates in orders_df and payments_df are standardized to a consistent format (YYYY-MM-DD) and any invalid dates are replaced with a default placeholder date (1970-01-01).


# Phase 3: Load

In [5]:
# Delete all records from Payments, Reviews, Orders, Users, and Products tables
tables_to_clear = ['Payments', 'Reviews', 'Orders', 'Users', 'Products']
for table in tables_to_clear:
    cursor.execute(f"DELETE FROM {table}")
conn.commit()

# Insert data into Products table
product_id_mapping = {}
for index, row in products_df.iterrows():
    cursor.execute("""
        INSERT INTO Products (ProductName, Description, Price, StockQuantity, Category)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['ProductName'], row['Description'], row['Price'], row['StockQuantity'], row['Category']))
    product_id_mapping[index] = cursor.lastrowid
conn.commit()

# Insert Users Data and Map Old UserIDs to New UserIDs
user_id_mapping = {}
for index, row in users_df.iterrows():
    cursor.execute("""
        INSERT INTO Users (UserName, Email, Address, Password)
        VALUES (%s, %s, %s, %s)
    """, (row['UserName'], row['Email'], row['Address'], row['Password']))
    # Mapping old UserID index to the new UserID
    user_id_mapping[index + 1] = cursor.lastrowid
conn.commit()


# Update Orders Data with New UserIDs
orders_df['UserID'] = orders_df['UserID'].map(user_id_mapping)


# Insert data into Orders table
order_id_mapping = {}
for index, row in orders_df.iterrows():
    cursor.execute("""
        INSERT INTO Orders (UserID, OrderDate, TotalAmount)
        VALUES (%s, %s, %s)
    """, (row['UserID'], row['OrderDate'], row['TotalAmount']))
    order_id_mapping[index] = cursor.lastrowid
conn.commit()


# Map the new OrderIDs to payments_df
payments_df['OrderID'] = payments_df.index.map(order_id_mapping)



# Insert data into Payments table
for index, row in payments_df.iterrows():
    cursor.execute("""
        INSERT INTO Payments (OrderID, PaymentMethod, PaymentDate, Amount)
        VALUES (%s, %s, %s, %s)
    """, (row['OrderID'], row['PaymentMethod'], row['PaymentDate'], row['Amount']))
conn.commit()

# Map the new UserIDs and ProductIDs to reviews_df
reviews_df['UserID'] = reviews_df['UserID'].map(user_id_mapping)
reviews_df['ProductID'] = reviews_df.index.map(product_id_mapping)

# Filter out rows with invalid ratings
reviews_df['Rating'] = pd.to_numeric(reviews_df['Rating'], errors='coerce')
reviews_df = reviews_df.dropna(subset=['Rating', 'ProductID', 'UserID'])


# Insert data into Reviews table
for index, row in reviews_df.iterrows():
    cursor.execute("""
        INSERT INTO Reviews (ProductID, UserID, Rating, ReviewText)
        VALUES (%s, %s, %s, %s)
    """, (row['ProductID'], row['UserID'], row['Rating'], row['ReviewText']))
conn.commit()

## What’s Happening Here?
* Clearing Existing Data: Before inserting new data, we delete all records from the target tables (Payments, Reviews, Orders, Users, and Products) to avoid conflicts and ensure we’re working with fresh data.
* Inserting Data: We iterate through each DataFrame, row by row, and insert the data into the corresponding table in the MySQL database. The cursor.execute method is used to run SQL INSERT statements with the data from each row.
* Mapping IDs: As we insert data, we map old IDs to new ones. For instance, the UserID in the Orders table is updated to match the new UserID in the Users table after insertion.
* Filtering and Loading Reviews: We filter out any invalid reviews (e.g., those with missing ratings) before inserting them into the Reviews table.

## Final Verification

In [7]:
# Final Verification
def verify_table_count(table_name, expected_count):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    if count == expected_count:
        print(f"{table_name} verification passed: {count} records.")
    else:
        print(f"{table_name} verification failed: Expected {expected_count}, but found {count}.")

# Verify Products table
verify_table_count('Products', len(products_df))

# Verify Users table
verify_table_count('Users', len(users_df))

# Verify Orders table
verify_table_count('Orders', len(orders_df))

# Verify Payments table
verify_table_count('Payments', len(payments_df))

# Verify Reviews table
verify_table_count('Reviews', len(reviews_df))

Products verification passed: 60 records.
Users verification passed: 10 records.
Orders verification passed: 60 records.
Payments verification passed: 60 records.
Reviews verification passed: 51 records.


## What’s Happening Here?
* We define a verify_table_count function that checks the number of records in each table and compares it to the expected count from the DataFrames.
* This ensures that all data has been correctly inserted into the database and that no records were missed.
